In [14]:
members = ['M','C','Aa','An','Ar','Ab']
expenses = [ 'Auto1 M 75 Aa 55, equal',
            'Local1 C 260 An 110, equal',
            'Auto2 M 450 C 450, equal',
            'Chai Ab 80, equal',
            'Maggi C 200, equal',
            'Poha M 80, equal',
            'Bottle M 10 C 50, equal',
            'Juice M 120, equal',
            'Auto3 C 600, equal',
            'Local2 M 110, equal',
            'Lunch Aa 1186, equal',
            'Auto4 M 60 C 60, equal',
            'Shopping M 202 C 55 An 15 Ar 173 Ab 55, equal',
            'Torch Ar 450, 150 0 150 0 0 150']

In [15]:
import pandas as pd
import numpy as np

# spenders_data, consumers_data
# spendersTotal, consumersTotal
# net
# toPay, balance

In [16]:
def preProcess(expense):
    event_data = expense.split(',')
    event = event_data[0].split(' ')[0]
    spent = event_data[0].split(' ')[1:]
    consumed = event_data[1].split(' ')[1:]

    ## Change consumed data based on equal distribution or not
    if len(consumed)==1 and consumed[0]=='equal':
        total = np.sum(np.array(spent[1::2],dtype=float))
        consumed = [total/len(members)]*len(members)
    else:
        consumed = np.array(consumed,dtype=float)

    return event,spent,consumed

def addConsumersEvent(consumers_data,event,consumed):
    ## Set Consumed resources row
    consumers_data.loc[len(consumers_data.index)] = consumed

    ## Set index
    d_index = list(consumers_data.index)
    d_index[len(consumers_data.index)-1] = event
    consumers_data.index = d_index
    return consumers_data

def addSpendersEvent(spenders_data,event,spent):
    spenders = spent[::2]
    spent = spent[1::2]
    ## Make zero row
    spenders_data.loc[len(spenders_data.index)] = np.zeros(len(spenders_data.columns))
    
    ## Edit respective cols to make the spenders row
    for i,name in enumerate(spenders):
        spenders_data[name][len(spenders_data.index)-1] = spent[i]
        
    ## Set index
    d_index = list(spenders_data.index)
    d_index[len(spenders_data.index)-1] = event
    spenders_data.index = d_index

    return spenders_data


In [17]:
consumers_data = pd.DataFrame(columns=members)
spenders_data = pd.DataFrame(columns=members)

for i in range(len(expenses)):
    event,spent,consumed = preProcess(expenses[i])
    # Add consumers event
    consumers_data = addConsumersEvent(consumers_data,event,consumed)
    # Add spenders event
    spenders_data = addSpendersEvent(spenders_data,event,spent)

spendersTotal = spenders_data.sum() 
consumersTotal = consumers_data.sum()
net = spendersTotal-consumersTotal


In [18]:
# print(net)
# print(spenders_data)
# print(consumers_data)

In [19]:
toPay = -net[net<0].copy().sort_values()
balance = net[net>=0].copy().sort_values(ascending=False)
# print(balance)
# print(toPay)

In [20]:
for i, amt in enumerate(toPay):
    while toPay[i]>0.2:
        for j,bal in enumerate(balance):
            if balance[j]==0: continue
            if toPay[i]<balance[j]:
                # pay everything and break bal for loop
                print(f'{toPay.keys()[i]} should pay Rs {toPay[i]} to {balance.keys()[j]}.')
                balance[j] -= toPay[i]
                toPay[i] = 0
                # print(f'Now {toPay.keys()[i]} has Rs {toPay[i]} to pay. {balance.keys()[j]} is yet to receive {balance[j]}')
                break
            elif toPay[i]>=balance[j]: # amt>bal
                # pay bal and move to next lender
                print(f'{toPay.keys()[i]} should pay Rs {balance[j]} to {balance.keys()[j]}.')
                toPay[i] -= balance[j]
                balance[j] = 0
                # print(f'Now {toPay.keys()[i]} is yet to pay Rs {toPay[i]} to someone. And balance with {balance.keys()[j]} is {balance[j]}.')


Ab should pay Rs 757.6666666666667 to C.
An should pay Rs 174.66666666666652 to C.
An should pay Rs 348.33333333333326 to Aa.
An should pay Rs 94.66666666666697 to M.
Ar should pay Rs 119.66666666666629 to M.


In [21]:
consumersTotal

M     892.666667
C     742.666667
Aa    892.666667
An    742.666667
Ar    742.666667
Ab    892.666667
dtype: float64

In [22]:
consumers_data.round(1).sort_values(by='M',ascending=False)

,M,C,Aa,An,Ar,Ab
Lunch,197.7,197.7,197.7,197.7,197.7,197.7
Auto2,150.0,150.0,150.0,150.0,150.0,150.0
Torch,150.0,0.0,150.0,0.0,0.0,150.0
Auto3,100.0,100.0,100.0,100.0,100.0,100.0
Shopping,83.3,83.3,83.3,83.3,83.3,83.3
Local1,61.7,61.7,61.7,61.7,61.7,61.7
Maggi,33.3,33.3,33.3,33.3,33.3,33.3
Auto1,21.7,21.7,21.7,21.7,21.7,21.7
Juice,20.0,20.0,20.0,20.0,20.0,20.0
Auto4,20.0,20.0,20.0,20.0,20.0,20.0


In [23]:
spenders_data

,M,C,Aa,An,Ar,Ab
Auto1,75.0,0.0,55.0,0.0,0.0,0.0
Local1,0.0,260.0,0.0,110.0,0.0,0.0
Auto2,450.0,450.0,0.0,0.0,0.0,0.0
Chai,0.0,0.0,0.0,0.0,0.0,80.0
Maggi,0.0,200.0,0.0,0.0,0.0,0.0
Poha,80.0,0.0,0.0,0.0,0.0,0.0
Bottle,10.0,50.0,0.0,0.0,0.0,0.0
Juice,120.0,0.0,0.0,0.0,0.0,0.0
Auto3,0.0,600.0,0.0,0.0,0.0,0.0
Local2,110.0,0.0,0.0,0.0,0.0,0.0
